In [1]:
import pymysql
import pandas as pd
from tqdm import tqdm

In [2]:
def GetMedication(disease,path="/Users/sinianzhang/Downloads"):
    database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
    cursor = database.cursor()
    
    ## Find Cui of Disease
    command = '''select distinct cui
                from mrconso
                where str = %s and lat="ENG";'''      
    cursor.execute(command,disease)
    cui = cursor.fetchone()[0]
    
    ## Find the Medication
    command = '''select cui2,mrconso.str
                from mrrel,mrconso
                where cui1 in (select cui2 from mrrel where cui1= %s and rela="mapped_to")  and rela = "may_treat" and mrconso.cui=cui2 and mrconso.lat="ENG";'''
                
    cursor.execute(command,cui)
    allmedication = cursor.fetchall()
    
    allmedication = pd.DataFrame(allmedication,columns=["cui","medication"])
    
    ## save the result
    allmedication.to_csv(path+"/" + "medication of " + disease + ".csv",encoding="utf_8_sig",index=False)
    
    cursor.close()
    database.close()

In [ ]:
GetMedication("Rheumatoid Arthritis")

In [5]:
def GetMedicationClass(data,path="/Users/sinianzhang/Downloads"):

    data = data.groupby('cui')['medication'].first()
    data = pd.DataFrame(data).reset_index()
    
    database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
    cursor = database.cursor()
    
    command = '''SELECT DISTINCT mrrel.cui1, mrconso.str
    FROM mrrel, mrconso
    WHERE mrrel.cui2 = %s AND mrrel.rela = 'isa' AND mrconso.cui = mrrel.cui1 AND mrconso.lat = 'ENG';'''

    ClassData = None

    for index,row in tqdm(data.iterrows()):
        
        cursor.execute(command,row["cui"])
        result = cursor.fetchall()
        Temp = pd.DataFrame(result,columns=["ClassCui","ClassStr"])
        Temp.insert(0, "MedicationStr", [row["medication"]]*len(Temp), allow_duplicates=True)
        Temp.insert(0, "MedicationCui", [row["cui"]]*len(Temp), allow_duplicates=True)
        ClassData = pd.concat([ClassData,Temp],ignore_index=True)
        
    ClassData.to_csv(path+"/" + "medication class.csv",encoding="utf_8_sig",index=False)
    
    cursor.close()
    database.close()

In [7]:
data = pd.read_csv("/Users/sinianzhang/Downloads/medication of Rheumatoid Arthritis.csv")

In [8]:
GetMedicationClass(data)

86it [59:37, 41.60s/it]


In [19]:
RAMedication = pd.read_csv("/Users/sinianzhang/Downloads/medication class.csv")

In [20]:
RAMedication = RAMedication.drop_duplicates(subset=['MedicationCui', 'ClassCui'])

In [21]:
RAMedication

,MedicationCui,MedicationStr,ClassCui,ClassStr
0,C0004057,aspirin,C0002771,Analgesics
30,C0004057,aspirin,C0003211,"Anti-Inflammatory Agents, Non-Steroidal"
73,C0004057,aspirin,C0003419,Antipyretics
87,C0004057,aspirin,C0004057,aspirin
132,C0004057,aspirin,C0016018,Fibrinolytic Agents
...,...,...,...,...
10420,C4726929,upadacitinib,C4726929,upadacitinib
10426,C4741328,rituximab-abbs,C0393022,rituximab
10439,C5195655,rituximab-pvvr,C0393022,rituximab
10452,C5222083,infliximab-axxq,C0666743,infliximab
